# 1. 自动识别文件

In [1]:
import os
import datetime

In [2]:
list_file = list(os.walk('./'))[0][2]
for file in list_file:
    if 'Lennon_整理好的替代料总表' in file:
        alternate = file
    elif '改价表' in file:
        stock = file
    elif 'US MRP' in file:
        mrp = file

print(f'alternate: \t{alternate}')
print(f'stock: \t\t{stock}')
print(f'mrp: \t\t{mrp}')

alternate: 	Lennon_整理好的替代料总表_20231122_174637.xlsx
stock: 		改价表 11-22.xlsx
mrp: 		US MRP20231120-运营.xlsx


# 2. 计算库存
## 2.1 替代料库存

In [3]:
import pandas as pd

In [4]:
df_alternate = pd.read_excel(alternate,
                             header=0).fillna('')
df_alternate

,主料SKU,替换料1,替换料2,替换料3,替换料4,替换料5,替换料6
0,USCA171876-3,USCA171876D-3,USCA171876,USCA171876D,,,
1,USWL02211,USWL02211H,,,,,
2,USTHA7040P,USTHA7040PX,,,,,
3,USBMR3204K,USBMR3204Y,,,,,
4,USDLA46848,USDLA46848G,,,,,
...,...,...,...,...,...,...,...
46253,USWR41583-C,USWR41583S-C1,USWR41583S-C,USWR41583,,,
46254,USHBZ15064-2-C,USHB15064-2-C,USHBZ15064-2,,,,
46255,USALT11462G-C,USALT11462-C,USALT11462A-C,USALT11462,,,
46256,USWR52350-C,USWR52350,,,,,


In [5]:
df_alternate['row'] = [str(i) for i in range(len(df_alternate))]

df_alternate

,主料SKU,替换料1,替换料2,替换料3,替换料4,替换料5,替换料6,row
0,USCA171876-3,USCA171876D-3,USCA171876,USCA171876D,,,,0
1,USWL02211,USWL02211H,,,,,,1
2,USTHA7040P,USTHA7040PX,,,,,,2
3,USBMR3204K,USBMR3204Y,,,,,,3
4,USDLA46848,USDLA46848G,,,,,,4
...,...,...,...,...,...,...,...,...
46253,USWR41583-C,USWR41583S-C1,USWR41583S-C,USWR41583,,,,46253
46254,USHBZ15064-2-C,USHB15064-2-C,USHBZ15064-2,,,,,46254
46255,USALT11462G-C,USALT11462-C,USALT11462A-C,USALT11462,,,,46255
46256,USWR52350-C,USWR52350,,,,,,46256


In [6]:
df_stock = pd.read_excel(stock,
                         header=0,
                         names=['SKU', '当日库存'],
                         usecols='A, F')

df_stock

,SKU,当日库存
0,USOP64305,3
1,USOP25053,0
2,USSGPL002,0
3,USOP97704,0
4,USOP97705,0
...,...,...
288676,USACC78341-C,1
288677,USTSS90412-C,1
288678,USAEV39292,1
288679,USFP9110M0-C,1


In [7]:
from tqdm import tqdm

In [8]:
for i in tqdm(range(len(df_stock)), desc='运行中', ncols=77):
    if df_stock.loc[i, 'SKU'].endswith('-NC'):
        df_stock = df_stock.drop(i)

运行中: 100%|█████████████████████| 288681/288681 [00:20<00:00, 13927.90it/s]


In [9]:
df_stock = df_stock.reset_index(drop=True)
df_stock['SKU'] = [df_stock.loc[i, 'SKU'][:-2] if df_stock.loc[i, 'SKU'].endswith('-N') else df_stock.loc[i, 'SKU'] for i in range(len(df_stock))]
df_stock = df_stock.groupby([df_stock['SKU']], as_index=False).sum()

df_stock

,SKU,当日库存
0,BPD8550086500+BRS31318,1
1,CAFP9183M0,1
2,CAWR41163,1
3,CS20230609,0
4,CS230530,0
...,...,...
88783,YHTWS58202-Y-6,0
88784,YHTWS5W1ZBA-Y-10,0
88785,YHTWS5W1ZBA-Y-8,0
88786,YHTWS802AA-Y-6,0


In [10]:
df_alternate['当日库存'] = 0
df_alternate = pd.merge(left=df_alternate, right=df_stock, how='left', left_on='主料SKU', right_on='SKU', suffixes=('', '_0')).fillna(0)
print('stock - 主料SKU')
df_alternate = pd.merge(left=df_alternate, right=df_stock, how='left', left_on='替换料1', right_on='SKU', suffixes=('', '_1')).fillna(0)
print('stock - 替换料1')
df_alternate = pd.merge(left=df_alternate, right=df_stock, how='left', left_on='替换料2', right_on='SKU', suffixes=('', '_2')).fillna(0)
print('stock - 替换料2')
df_alternate = pd.merge(left=df_alternate, right=df_stock, how='left', left_on='替换料3', right_on='SKU', suffixes=('', '_3')).fillna(0)
print('stock - 替换料3')
df_alternate = pd.merge(left=df_alternate, right=df_stock, how='left', left_on='替换料4', right_on='SKU', suffixes=('', '_4')).fillna(0)
print('stock - 替换料4')
df_alternate = pd.merge(left=df_alternate, right=df_stock, how='left', left_on='替换料5', right_on='SKU', suffixes=('', '_5')).fillna(0)
print('stock - 替换料5')
df_alternate = pd.merge(left=df_alternate, right=df_stock, how='left', left_on='替换料6', right_on='SKU', suffixes=('', '_6')).fillna(0)
print('stock - 替换料6')

df_alternate['当日库存'] = df_alternate['当日库存_0'] + df_alternate['当日库存_1'] + df_alternate['当日库存_2'] + df_alternate['当日库存_3'] + df_alternate['当日库存_4'] + df_alternate['当日库存_5'] + df_alternate['当日库存_6']
df_alternate = df_alternate[['主料SKU',
                             '替换料1',
                             '替换料2',
                             '替换料3',
                             '替换料4',
                             '替换料5',
                             '替换料6',
                             '当日库存']]

df_alternate

stock - 主料SKU
stock - 替换料1
stock - 替换料2
stock - 替换料3
stock - 替换料4
stock - 替换料5
stock - 替换料6


,主料SKU,替换料1,替换料2,替换料3,替换料4,替换料5,替换料6,当日库存
0,USCA171876-3,USCA171876D-3,USCA171876,USCA171876D,,,,11.0
1,USWL02211,USWL02211H,,,,,,21.0
2,USTHA7040P,USTHA7040PX,,,,,,22.0
3,USBMR3204K,USBMR3204Y,,,,,,12.0
4,USDLA46848,USDLA46848G,,,,,,16.0
...,...,...,...,...,...,...,...,...
46253,USWR41583-C,USWR41583S-C1,USWR41583S-C,USWR41583,,,,5.0
46254,USHBZ15064-2-C,USHB15064-2-C,USHBZ15064-2,,,,,32.0
46255,USALT11462G-C,USALT11462-C,USALT11462A-C,USALT11462,,,,15.0
46256,USWR52350-C,USWR52350,,,,,,0.0


In [11]:
list_df = []
for i in ['主料SKU', '替换料1', '替换料2', '替换料3', '替换料4', '替换料5', '替换料6']:
    df_temp =  df_alternate[[i,
                             '当日库存']]
    df_temp.columns = ['SKU',
                       '当日库存']
    list_df.append(df_temp)

df_position = pd.concat(list_df)
df_position = df_position[df_position['SKU'] != ''].reset_index(drop=True)

df_position

,SKU,当日库存
0,USCA171876-3,11.0
1,USWL02211,21.0
2,USTHA7040P,22.0
3,USBMR3204K,12.0
4,USDLA46848,16.0
...,...,...
99668,USGS6310,0.0
99669,USIC2A367A,0.0
99670,USIC71643Z,2.0
99671,USIC02680I-6,34.0


## 2.2 MRP表库存

In [12]:
df_mrp = pd.read_excel(mrp,
                       header=1,
                       usecols='B').fillna(0)

df_mrp

,MRP SKU
0,USTCV17215HIE-2-C
1,USET87039H-C
2,USOBDTK100
3,USBDA04108U-2-C
4,USBDA04108U-C
...,...
63377,USGS4917L8R-C
63378,USIC677650-8-C
63379,USDS86070-C
63380,USTLA48075


In [13]:
df_mrp = pd.merge(left=df_mrp, right=df_stock, how='left', left_on='MRP SKU', right_on='SKU').fillna(0)
df_mrp = pd.merge(left=df_mrp, right=df_position, how='left', left_on='MRP SKU', right_on='SKU', suffixes=('_stock', '_position')).fillna('')

df_mrp['库存'] = [df_mrp.loc[i, '当日库存_position'] if df_mrp.loc[i, '当日库存_position'] != '' else df_mrp.loc[i, '当日库存_stock'] for i in range(len(df_mrp))]
df_mrp = df_mrp[['MRP SKU',
                 '库存']]

df_mrp

,MRP SKU,库存
0,USTCV17215HIE-2-C,32.0
1,USET87039H-C,50.0
2,USOBDTK100,278.0
3,USBDA04108U-2-C,104.0
4,USBDA04108U-C,45.0
...,...,...
63377,USGS4917L8R-C,5.0
63378,USIC677650-8-C,22.0
63379,USDS86070-C,0.0
63380,USTLA48075,2.0


# 3. 保存

In [14]:
import datetime

In [15]:
writer = pd.ExcelWriter('Lennon_整理好的库存表_' + datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + '.xlsx')
df_mrp.to_excel(writer, index=False)
writer.close()